# Regression Analysis: 실습 프로젝트

In [ ]:
# 실습에서 사용할 모듈을 import 합니다.
# 이후의 셀을 실행하기 전에 항상 먼저 실행되어야 합니다. 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

# IPYthon 상에서 matplotlib 라이브러리가 
# 별도의 창 없이 즉시 plot을 출력할 수 있도록 설정을 변경합니다.
%config InlineBackend.figure_format = 'png'
%matplotlib inline

## 데이터 불러오기

`pandas` 라이브러리의 `read_csv` 함수를 통해 `.csv` 형식의 데이터를 불러올 수 있습니다. 불러온 데이터는 `pandas.DataFrame`의 행렬 형태로 반환됩니다. 데이터의 각 행(row)은 하나의 sample을 나타내고, 데이터의 각 열(column)은 ID, feature, 또는 예측하고자 하는 target이 될 수 있습니다. "House Prices" dataset에서는 `Id` 필드와 `SalePrice` 필드(target)를 제외한 `MSSubClass ~ SaleCondition` 사이의 열이 feature가 됩니다. 각 feature는 다양한 type을 가질 수 있으며, 크게는 numerical feature와 categorical feature로 구분됩니다. 필요한 경우, `data/데이터_명세.pdf` 파일을 참조하여 각 feature의 의미를 확인하실 수 있습니다. 

한편, "House Prices" dataset은 2개의 데이터로 분리되어 있습니다:
- Training data: 실제로 분석을 수행하고, regression 모델을 훈련시키는 데 사용되는 데이터입니다.
- Testing data: 훈련된 모델의 성능을 평가하기 위한 용도로 사용됩니다. 모델의 훈련 과정 도중에 노출되지 않아야 합니다.


In [ ]:
# 데이터를 불러옵니다.
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

# 데이터 전처리 과정을 위해 두 데이터를 임시적으로 합치되, ID와 target field를 제외합니다.
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

In [ ]:
# Training data의 처음 일부분을 확인할 수 있습니다.
train.head()

## 데이터 전처리

대개의 경우, 데이터의 분석 과정은 **"Garbage in, garbage out"**을 따릅니다: 좋은 분석을 얻기 위해서는 데이터의 quality 또한 좋아야 합니다.

Real-world의 데이터는 수집 과정에서 발생하는 다양한 요인에 의해 즉시 분석하기 어려울 때가 많습니다:
- Missing values
- Impossible values, combinations (e.g. Sex: Male, Pregnant: Yes)
- Redundant/Noisy features
- ... etc.

데이터 전처리 과정은 본격적인 분석 이전에 이러한 문제에 대응하여 수행하는 과정 전체를 의미합니다. 본 실습에서는 전처리 과정 이후의 분석에 조금 더 집중할 것이기 때문에, 이 과정에 대해 상세히 다루지는 않습니다. 그렇지만 조금 더 의미있는 분석을 위해, 다음과 같은 기본적인 전처리를 수행할 것입니다:
- Numeric feature 중 너무 한 쪽으로 분포가 치우친("skewed") feature의 경우 `log(feature + 1)`를 취합니다 - feature를 조금 더 normal 한 형태로 만들어 줍니다.
- Categorical feature를 여러 개의 binary numerical feature로 변환시킵니다. (dummy variables)
- Data에 존재하는 missing value를 해당 column의 평균으로 채웁니다.

In [ ]:
# Target의 분포와 log(feature + 1) 변환 이후의 히스토그램을 출력합니다.
# 변환 이후의 분포가 더 대칭적인 형태를 띈다는 점에 주목하십시오.
matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)
prices = pd.DataFrame({"0_price":train["SalePrice"], "1_log(price + 1)":np.log1p(train["SalePrice"])})
prices.hist()

In [ ]:
# Target에 대해 log(feature + 1) 변환을 취합니다. 
train["SalePrice"] = np.log1p(train["SalePrice"])

# Numeric feature 중에서도 치우친 feature가 있다면 마찬가지의 변환을 수행합니다.
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

# Dummy variable을 통해 categorical feature를 numeric feature로 변환합니다.
all_data = pd.get_dummies(all_data)

# 평균을 이용해 missing values를 처리합니다.
all_data = all_data.fillna(all_data.mean())

# 데이터가 최종적으로 어떻게 전처리 되었는지 원래 데이터와 비교해보세요.
all_data.head()

In [ ]:
# scikit-learn의 알고리즘에 입력할 수 있는 형태의 데이터를 생성합니다.
# X_train에는 training에 필요한 데이터가, X_test에는 test에 필요한 데이터가 저장됩니다.
n_train = train.shape[0]
X_train, X_test = all_data[:n_train], all_data[n_train:]
y = train.SalePrice

## 회귀 모델 학습

이제 전처리된 데이터를 통해 새로운 데이터의 target을 예측하는 회귀 모델을 학습할 것입니다. '좋은' 회귀 모델을 학습한다는 것은, target을 모르는 '새로운' 데이터가 모델에 주어졌을 때 그것의 실제 target에 가까운 값을 예측할 수 있게끔 하는 것입니다. 이를 위해, 다음과 같은 질문을 고려할 수 있습니다:

- 어떠한 모델을 사용할 것인가?
- 모델의 hyperparameter는 어떻게 설정되어야 하는가?
- '좋은' 모델을 어떻게 평가할 것인가?

본 실습에서는, scikit-learn 모듈의 다양한 regression model을 사용해볼 것입니다. 모델을 평가하기 위해 cross-validation error를 측정할 것이고, 이를 통해 모델의 hyperparameter를 찾는 방법에 대해서도 알아볼 것입니다. 

아래에 예시로 주어지는 코드에서는 scikit-learn 모듈의 ordinary least squares와 Lasso model을 학습해보고, 둘을 비교하면서 $l_1$ regularization이 모델에 어떠한 영향을 주는지 확인합니다. 학습된 모델을 이용해 Testing data의 target을 실제로 예측해보고, 이것을 Kaggle에 (https://www.kaggle.com/c/house-prices-advanced-regression-techniques) 제출하여 test score를 확인할 수 있을 것입니다. 

주어진 방식을 토대로, 프로젝트를 수행하면서 다양한 모델을 실험해 보세요. 

In [ ]:
# 학습에 필요한 모델 및 함수를 import 합니다.
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import cross_val_score

아래에 정의되는 함수 `mean_cv_rmse()`는 주어진 모델이 학습하고자 하는 데이터에 대해 얼마나 좋은 성능을 낼 것인지에 대한 지표로 사용됩니다. 함수의 반환값은 항상 양수를 가지며, 0에 가까울 수록 더 적합한 모델임을 의미합니다. 모델을 평가하는 과정에서 발생할 수 있는 overfitting을 방지하기 위해, 본 실습에서는 **cross-validation (CV)**을 사용합니다. CV에 대한 자세한 설명은 다음 링크를 참조하시면 좋습니다: http://scikit-learn.org/stable/modules/cross_validation.html

In [ ]:
# 주어진 모델의 cross-validation root-mean-squared error (RMSE) score를 계산합니다.
# 이 함수에서는 5-fold cross-valiation을 수행하고, 5개의 score를 평균한 값이 반환됩니다.
# Cross-validation을 계산하기 위해, 학습 데이터를 사용해서 총 5번의 학습을 수행해야 합니다.
def mean_cv_rmse(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv=5))
    return(rmse.mean())

우선 가장 간단한 형태의 linear model인 ordinary least squares (OLS)를 학습해 봅시다. `scikit-learn`의 `LinearRegression` class를 이용하면 쉽게 OLS 모델을 학습할 수 있습니다. 학습을 수행하기 전에, `mean_cv_rmse()` 함수를 통해 OLS 모델의 예상 성능을 측정해볼 수 있습니다. OLS 모델의 cross-validation RMSE score는 대략 0.165로 계산될 것입니다. 

In [ ]:
# 새로운 OLS 모델을 정의합니다. 
model_ols = LinearRegression()

# OLS 모델의 예상 성능을 측정합니다. 
print("Score: %.4f" % mean_cv_rmse(model_ols))

# 모델을 전체 학습 데이터로 훈련시킵니다.
model_ols.fit(X_train, y)

그 다음으로, Lasso regression을 학습해볼 것입니다. Lasso regression은 기존 OLS 모델에서 $l_1$-regularization을 추가한 형태입니다. OLS 모델을 학습했던 방식과 비슷하게, scikit-learn의 `Lasso` class를 통해 학습할 수 있습니다. 이 class에 대한 자세한 정보는 http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html 를 참조하십시오. :
```
class sklearn.linear_model.Lasso(alpha=1.0, ...)
```
OLS 모델을 학습할 때와는 달리, Lasso에서는 추가적으로 설정해주어야 할 **hyperparameter `'alpha'`**가 있습니다. Lasso model에서 `alpha`는 모델이 얼마나 'sparse' 하게 학습될 것인지를 결정합니다. 학습하고자 하는 데이터가 많은 feature를 가지고 있는 경우, 이렇게 모델의 sparsity를 조절하는 것은 overfitting을 막는데 효과적일 수 있습니다. `alpha`가 큰 모델일수록, 더 많은 feature를 unimportant하다고 여기고 그 계수를 0으로 만드는 경향이 생깁니다. 하지만 과도하게 `alpha`가 클 경우, 모델의 표현력를 잃으면서 data를 정확히 capture하는 모델을 학습하지 못할 수도 있습니다. 따라서 학습하는 data에 적합한 `alpha`를 찾는 것이 중요합니다.

최적의 `alpha`를 찾기 위해, 여기서는 또다시 cross-validation score를 이용합니다. 여러 `alpha`의 후보에 대해 가장 뛰어난 cross-validation score를 기록한 `alpha`를 취하는 방식으로 Lasso의 hyperparameter를 결정할 것입니다. Lasso의 특성 상, `alpha`의 후보는 log scale에서 동등한 간격을 가지게끔 선택되었습니다.

In [ ]:
# alpha의 후보를 설정합니다. 
# Lasso의 특성 상, log-scale에서 값들이 균일하도록 후보를 설정했습니다.
alphas = np.logspace(start=-5, stop=0, num=10)
print("Candidates of alpha = \n%s" % alphas)

# 각각의 alpha에 대한 모델 성능을 측정합니다. 
cv_lasso = [mean_cv_rmse(Lasso(alpha = alpha)) for alpha in alphas]

# alpha에 따른 성능 측정 결과를 그래프로 나타냅니다.
# alpha는 log scale로 표현되었습니다.
cv_lasso = pd.Series(cv_lasso, index=alphas)
cv_lasso.plot(logx=True)
plt.xlabel("log(alpha)")
plt.ylabel("RMSE")

In [ ]:
best_alpha = cv_lasso.idxmin()
print("Best alpha: %f, score: %f" % (best_alpha, cv_lasso.min()))

측정 결과로 그려진 U자 형태의 그래프를 통해, `alpha`의 효과를 다시 한 번 확인할 수 있습니다: 
- `alpha`가 계속 작아질 경우, Lasso는 OLS와 같아집니다. 
- `alpha`가 너무 커질 경우, 모델의 표현력을 잃을 수 있습니다. 

여기서는 `alpha = 0.000464` 에서 가장 좋은 성능을 보입니다. 우리는 이 값을 Lasso 모델을 학습할 때 사용할 것입니다. 
Lasso에서 측정된 성능과 이전 OLS 모델의 성능을 비교해 보세요.

In [ ]:
# 가장 좋은 성능을 보였던 alpha를 이용해 모델을 정의하고, 전체 데이터로 다시 학습합니다.
model_lasso = Lasso(alpha=best_alpha)
model_lasso.fit(X_train, y)

### **Q1.** 
**비슷한 방식으로, Ridge regression 모델을 학습해보세요. Ridge regression은 기존 OLS 모델에서 $l_2$-regularization을 추가한 형태이고, scikit-learn의 `Ridge` class를 통해 학습할 수 있습니다. 이 class에 대한 자세한 정보는 http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html 를 참조하십시오. :**
```
class sklearn.linear_model.Ridge(alpha=1.0, ...)
```
**아래 주어진 스켈레톤 소스코드에서 출발하여 최적의 `alpha`를 찾고, 전체 데이터를 이용하여 훈련시키는 과정을 수행하시면 됩니다.**

In [ ]:
# Ridge 모델을 import 합니다.
from sklearn.linear_model import Ridge

# alpha의 후보를 설정합니다. 
# Ridge의 특성 상, uniform-scale에서 후보를 설정했습니다.
alphas = np.linspace(start=1.0, stop=30.0, num=10)
print("Candidates of alpha = \n%s" % alphas)

############## YOUR CODE HERE BELOW ##############


## 모델 해석하기

선형 모델이 가진 강력한 장점은, 학습된 모델의 **해석 가능성**(interpretability), 즉, 모델의 input feature를 통해 output을 설명할 수 있다는 점입니다. 선형 모델보다 더 좋은 성능을 기대할 수 있는 머신러닝 알고리즘은 많이 있지만, 학습된 모델을 다시 해석하여 그 의미를 설명할 수 있는 알고리즘은 드뭅니다. 여기서는 위에서 학습한 Lasso 모델을 예시로 들어, linear model을 해석하는 방법에 대해 알아보겠습니다. 

일반적으로, linear model에서는 각 feature에 대응하는 coefficient를 학습합니다. Linear model의 해석은 주로 해당 모델이 학습한 coefficient를 관찰하면서 이루어집니다.

In [ ]:
coef_lasso= pd.Series(model_lasso.coef_, index = X_train.columns)

간단한 예시로, 우리는 Lasso model의 non-zero coefficient를 세어볼 수 있습니다. 어떤 feature의 coefficient가 0이라는 것은 Lasso model이 해당 feature를 중요하지 않다고 판단하고 prediction에서 전혀 사용하지 않았다는 것을 의미합니다. 이렇게, 학습된 Lasso 모델은 전체 feature 가운데 오직 113개만 사용했다는 것을 확인할 수 있습니다.

In [ ]:
print("The Lasso model picked " + str(sum(coef_lasso != 0)) + " variables" +
      "and eliminated the other " +  str(sum(coef_lasso == 0)) + " variables")

### **Q2.** 
**위에서는 Lasso 모델의 non-zero coefficient를 세어 보았습니다. 그렇다면, OLS를 이용한 모델 (`model_ols`)에서는 몇 개의 non-zero coefficient가 있나요? Ridge 모델의 경우는 어떤가요? 결과를 비교했을 때, Lasso 모델에서 관찰되는 특징은 무엇인가요?**

In [ ]:
############## YOUR CODE HERE BELOW ##############

한편, coefficient의 절댓값이 큰 feature를 주목할 수도 있습니다. coefficient의 절댓값이 크다는 것은, prediction에 해당 feature가 큰 영향을 준다는 것과 연관되어 있습니다. 이렇게 데이터의 important features를 선택하는 경우에도 linear model은 의미있는 정보를 제공합니다.

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)

# 절댓값이 큰 coefficients를 선택합니다.
imp_coef = pd.concat([coef_lasso.sort_values().head(10), 
                      coef_lasso.sort_values().tail(10)])
imp_coef.plot(kind = "barh")
plt.title("Important features in the Lasso Model")

### **Q3.** 
**위 분석에 따르면, house price 예측 모델에서 높은 가격의 가장 큰 요인은 (feature는) 무엇인가요? 반면, 가장 가격을 떨어뜨렸던 요인은 무엇인가요? 배포해드린 데이터 명세 자료를 바탕으로, 각 feature가 실제로 가지는 의미를 분석해주세요. 이 밖에도, 추가적으로 자유롭게 결과를 분석해주시길 바랍니다.**

## 모델 적용하기

학습된 모델은 새로운 데이터에 대해서도 prediction을 수행할 수 있습니다. 이제 Testing data에 대해서도 prediction을 해 보고, 결과를 Kaggle에 제출해봅시다. 모델을 학습하기 전에 target을 `log(feature+1)` 로 변환했던 것을 유의하시길 바랍니다. `numpy.expm1()` 함수를 통해 역변환을 수행하여 실제 target의 scale에 맞추어 줍니다.

In [ ]:
# OLS model과 Lasso model의 prediction을 계산합니다.
ols_preds = np.expm1(model_ols.predict(X_test))
lasso_preds = np.expm1(model_lasso.predict(X_test))

print(ols_preds)
print(lasso_preds)

In [ ]:
# prediction 결과를 Kaggle에 제출할 수 있는 포맷으로 저장합니다.
solution = pd.DataFrame({"id":test.Id, "SalePrice":lasso_preds})
solution.to_csv("house_sol_lasso.csv", index = False)

solution = pd.DataFrame({"id":test.Id, "SalePrice":ols_preds})
solution.to_csv("house_sol_ols.csv", index = False)

결과를 제출한 뒤, training 데이터에서 더 좋은 성능을 보였던 Lasso 모델의 결과가 실제 test 데이터에서도 잘 나오는지 Kaggle 점수를 통해 확인해봅시다.
https://www.kaggle.com/c/house-prices-advanced-regression-techniques

### **Q4.** 
**이제 자유롭게 회귀 모델링을 수행하여 최대한 좋은 성능의 모델을 만들어 보세요. 성능을 올릴 수 있는 다양한 방법이 존재합니다:**

* 다른 regression 모델을 활용: [API document](http://scikit-learn.org/stable/modules/classes.html)
    * e.g. [Generalized linear models](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor), [Random forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor), [Neural network](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor)
* 여러 개의 모델 prediction을 평균하여 새로운 결과 도출: [앙상블 기법](https://swalloow.github.io/bagging-boosting) 
* 데이터를 새로운 방식으로 정제
* ... etc.

**다양한 모델의 prediction을 실제로 Kaggle에 제출하고, 가장 높은 test score가 나온 모델은 Kaggle 화면을 스크린샷으로 캡처하여 보고서에 첨부해주세요. Kaggle에 제출이 불가능한 경우, 해당 모델의 `mean_cv_rmse()` 점수를 대신 레포트 해주세요. 최종 test score는 보고서 점수에 일부 반영될 예정이지만, 성능이 오르지 않더라도 본인이 시도했던 방법들을 보고서에 상세히 작성해주세요.**

In [ ]:
############## YOUR CODE HERE BELOW ##############